In [1]:
!pip install keras-tuner

In [2]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [3]:
df= keras.datasets.fashion_mnist


In [4]:
#Split into images and labels
(train_images, train_labels),(test_images, test_labels)= df.load_data()

In [5]:
#scaling the images
train_images= train_images/ 255.0
test_images= test_images/ 255.0

In [6]:
#shape
train_images[0].shape

(28, 28)

In [7]:
test_images[0].shape

(28, 28)

In [8]:
train_images= train_images.reshape(len(train_images), 28,28,1)
test_images= test_images.reshape(len(test_images), 28,28,1)

In [9]:
def model_building(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=8, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=8, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=8, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-5])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model



In [12]:
tuner_search=RandomSearch(model_building,
                          objective='val_accuracy',
                          max_trials=6,directory='output',project_name="Fashion Mnist")
    

INFO:tensorflow:Reloading Oracle from existing project output/Fashion Mnist/oracle.json
INFO:tensorflow:Reloading Tuner from output/Fashion Mnist/tuner0.json


In [13]:
tuner_search.search_space_summary()

Search space summary
Default search space size: 6
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': None}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [14]:
tuner_search.search(train_images,train_labels,epochs=5,validation_split=0.2)

INFO:tensorflow:Oracle triggered exit


In [20]:
model=tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        32288     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2064496   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,099,034
Trainable params: 2,099,034
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.2, initial_epoch=5)

Epoch 6/10
1500/1500 [==============================] - 18s 11ms/step - loss: 0.0613 - accuracy: 0.9782 - val_loss: 0.3432 - val_accuracy: 0.9097
Epoch 7/10
1500/1500 [==============================] - 16s 11ms/step - loss: 0.0413 - accuracy: 0.9843 - val_loss: 0.3901 - val_accuracy: 0.9107
Epoch 8/10
1500/1500 [==============================] - 16s 11ms/step - loss: 0.0294 - accuracy: 0.9892 - val_loss: 0.4275 - val_accuracy: 0.9082
Epoch 9/10
1500/1500 [==============================] - 16s 11ms/step - loss: 0.0242 - accuracy: 0.9914 - val_loss: 0.4737 - val_accuracy: 0.9079
Epoch 10/10
1500/1500 [==============================] - 16s 11ms/step - loss: 0.0228 - accuracy: 0.9920 - val_loss: 0.4919 - val_accuracy: 0.9128
